In [1]:
# Reference: https://www.geeksforgeeks.org/twitter-sentiment-analysis-using-python/
# API: https://developer.twitter.com/en/apps

In [ ]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

In [1]:
twitter_api_key='XXXXX'
twitter_secret='XXXXX'
twitter_bearer='XXXXX'
twitter_access_token='XXXXX'
twitter_access_token_secret='XXXXX'

In [9]:
# necessary imports for the project
import tweepy as tp
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import string

from textblob import TextBlob
from datetime import datetime
from dotenv import load_dotenv

# constants
PATH = os.path.expanduser('~') + '/'
env_file = '.env'
today = datetime.today().strftime('%Y-%m-%d')

# read environment file
def env_reader(env_path):
    '''
    This function will read the environment variables given the location of where the
    .env file is located
    '''
    if not load_dotenv(env_path):
        return print('Environment File Not Found')
    return print('Loaded!')

env_reader(os.getcwd() + '/' + env_file)

# connect to twitter API
def connect_twitter(api_key, secret_key, access_token, secret_access_token):
    '''
    This function will create a connection to the twitter api using the necessary
    credentials associated to the project.
    
    params:
        api_key (String) : Taken from twitter developer account
        secret_key (String) : Taken from twitter developer account
        access_token (String) : Taken from twitter developer account
        secret_access_token (String) : Taken from twitter developer account
         
    returns:
        This function will return an API which can be called
    '''
    auth = tp.OAuthHandler(
        consumer_key = twitter_api_key, 
        consumer_secret = twitter_secret
    )
    auth.set_access_token(
        key =  twitter_access_token, 
        secret = twitter_access_token_secret
    )
    api = tp.API(auth)
    
    try:
        api.verify_credentials()
        print("Connection to Twitter established.")
    except:
        print("Failed to connect to Twitter.")
    return api

api = connect_twitter(
    api_key = os.getenv("twitter_api_key"), 
    secret_key = os.getenv("twitter_secret"),
    access_token = os.getenv("twitter_access_token"),
    secret_access_token = os.getenv("twitter_access_token_secret")
)

Loaded!
Failed to connect to Twitter.


In [10]:
import tweepy
consumer_key= 'zaJLhSe2PkUHPa4GqYvqdYT5O'
consumer_secret= 'hTMjnOfOiydzye7tvzUDHfoMJvgtz8Q1ZwX98GFvsiZpdxt6Ml'
access_token= '1258950814120374274-dhBaRO8KSExrfQ9ntVN33ttasMdL5W'
access_token_secret= 'GDTIwXlVjQKzJsmMfkZdOAaBFaveZiKZRRJjFSkpLes6q'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

In [8]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

class TwitterClient(object):
	'''
	Generic Twitter Class for sentiment analysis.
	'''
	def __init__(self):
		'''
		Class constructor or initialization method.
		'''
		# keys and tokens from the Twitter Dev Console
		consumer_key = 'zaJLhSe2PkUHPa4GqYvqdYT5O'
		consumer_secret = 'hTMjnOfOiydzye7tvzUDHfoMJvgtz8Q1ZwX98GFvsiZpdxt6Ml'
		access_token = '1258950814120374274-dhBaRO8KSExrfQ9ntVN33ttasMdL5W'
		access_token_secret = 'GDTIwXlVjQKzJsmMfkZdOAaBFaveZiKZRRJjFSkpLes6q'

		# attempt authentication
		try:
			# create OAuthHandler object
			self.auth = OAuthHandler(consumer_key, consumer_secret)
			# set access token and secret
			self.auth.set_access_token(access_token, access_token_secret)
			# create tweepy API object to fetch tweets
			self.api = tweepy.API(self.auth)
		except:
			print("Error: Authentication Failed")

	def clean_tweet(self, tweet):
		'''
		Utility function to clean tweet text by removing links, special characters
		using simple regex statements.
		'''
		return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

	def get_tweet_sentiment(self, tweet):
		'''
		Utility function to classify sentiment of passed tweet
		using textblob's sentiment method
		'''
		# create TextBlob object of passed tweet text
		analysis = TextBlob(self.clean_tweet(tweet))
		# set sentiment
		if analysis.sentiment.polarity > 0:
			return 'positive'
		elif analysis.sentiment.polarity == 0:
			return 'neutral'
		else:
			return 'negative'

	def get_tweets(self, query, count = 10):
		'''
		Main function to fetch tweets and parse them.
		'''
		# empty list to store parsed tweets
		tweets = []


		# call twitter api to fetch tweets
		fetched_tweets = self.api.search_tweets(q = query, count = count)

		# parsing tweets one by one
		for tweet in fetched_tweets:
			# empty dictionary to store required params of a tweet
			parsed_tweet = {}
			# saving text of tweet
			parsed_tweet['text'] = tweet.text
			# saving sentiment of tweet
			parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)

			# appending parsed tweet to tweets list
			if tweet.retweet_count > 0:
				# if tweet has retweets, ensure that it is appended only once
				if parsed_tweet not in tweets:
					tweets.append(parsed_tweet)
			else:
				tweets.append(parsed_tweet)

		# return parsed tweets
		return tweets



def main():
	# creating object of TwitterClient Class
	api = TwitterClient()
	# calling function to get tweets
	tweets = api.get_tweets(query = 'Donald Trump', count = 200)

	# picking positive tweets from tweets
	ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
	# percentage of positive tweets
	print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
	# picking negative tweets from tweets
	ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
	# percentage of negative tweets
	print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
	# percentage of neutral tweets
	print("Neutral tweets percentage: {} % \
		".format(100*(len(tweets) -(len( ntweets )+len( ptweets)))/len(tweets)))

	# printing first 5 positive tweets
	print("\n\nPositive tweets:")
	for tweet in ptweets[:10]:
		print(tweet['text'])

	# printing first 5 negative tweets
	print("\n\nNegative tweets:")
	for tweet in ntweets[:10]:
		print(tweet['text'])

if __name__ == "__main__":
	# calling main function
	main()


Forbidden: 403 Forbidden
453 - You currently have Essential access which includes access to Twitter API v2 endpoints only. If you need access to this endpoint, you’ll need to apply for Elevated access via the Developer Portal. You can learn more here: https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-leve

In [12]:
search_result = api.search_tweets(q="jakarta",count=100)

Forbidden: 403 Forbidden
453 - You currently have Essential access which includes access to Twitter API v2 endpoints only. If you need access to this endpoint, you’ll need to apply for Elevated access via the Developer Portal. You can learn more here: https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-leve